In [1]:
import subprocess
import multiprocessing as mp
from multiprocessing import Lock, Queue
import time
import random
import sys
import os
import errno
import shutil 
import subprocess
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import cma
from decimal import Decimal
from astropy import units as u
from poliastro.bodies import Earth, Sun
from poliastro.twobody import Orbit
from poliastro.plotting import plot
from poliastro.plotting import OrbitPlotter
from numpy import linalg as LA
from mpl_toolkits.mplot3d import Axes3D
import astropy.units as u
from astropy import time as ast_time
from poliastro import iod
from astropy.coordinates import solar_system_ephemeris
from poliastro.ephem import get_body_ephem
from astropy import time as ast_time
from lamberts_problem_solver import *
import atexit
import transfer_orbit_plotter as top

In [2]:
def save_porkchop(): 

    # open a file to save the launch_date, tof, delta v for all the combinations
    filename = '../output/porkchop.txt'
    try:
        os.remove(filename)
    except OSError:
        pass
    
    f = open(filename, "ab")    
  
    #np.savetxt(f , delimiter=' ', header='Launch Date| TOF | Delta V', newline='\n')
    for launch_date in range(2461587, 2463598):
        
        launch_date_jd = ast_time.Time(launch_date, format='jd', scale='utc')
        rr_earth, vv_earth = get_body_ephem("earth", launch_date_jd)
        tmp = []
        
        for tof in range(1,1200):
            arrival_date_jd = ast_time.Time(launch_date + tof, format='jd', scale='utc') 
            rr_mars, vv_mars = get_body_ephem("mars", arrival_date_jd)
            tof_units = tof*u.day
                #print(tof_units)
            (va, vb), = iod.lambert(Sun.k, rr_earth, rr_mars, tof*24*3600*u.s)
            final_v = va.to(u.km/u.s) - vv_earth.to(u.km/u.s)
            if LA.norm(final_v.value) < 10:
                tmp.append([launch_date, tof, LA.norm(final_v.value)])
        
        np.savetxt(f , tmp, fmt='%5.3f', delimiter=' ', newline='\n')
    
    f.close()
            
save_porkchop()   

In [1]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:',__name__)
    if hasattr(os, 'getppid'):  # only available on Unix
        print('parent process:',os.getppid())
    print('process id:',os.getpid())

def f(name):
    info('function f')
    print('hello',name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 5575
process id: 8404
function f
module name: __main__
parent process: 8404
process id: 8415
hello bob


In [15]:
from multiprocessing import Pool, TimeoutError
import time
import os
import numpy as np

def f(x):
     return [x, x*x+y]

if __name__ == '__main__':
    pool = Pool(processes=4)              # start 4 worker processes

    # print "[0, 1, 4,..., 81]"
    a = pool.map(f,1, range(10))
    print(a)
    filename = '../output/porkchop.txt'
    try:
        os.remove(filename)
    except OSError:
        pass
    f = open(filename, "ab")    
    np.savetxt(f , a, fmt='%5.3f', delimiter=' ', newline='\n', header='Launch Date| TOF | Delta V')            
    f.close()   
    

TypeError: 'int' object is not iterable